---
---
**<font color='#FF8C00'><center><big> _Буду рад копеечке)_ </center></font>**

[<center> ![smaller.png](https://www.donationalerts.com/img/brand/donationalerts.svg) </center>](https://www.donationalerts.com/r/politrees)

---
---
**<center> English version of this same notebook: https://colab.research.google.com/drive/1VOJkx6dHRHs-bxM2Skmc54D3RC9cJ107 </center>**

---
**<center> Чтобы быть в курсе всех обновлений и изменений в этом блокноте, рекомендую подписаться на мой [Телеграм-канал](https://t.me/pol1trees) </center>**

_<center> (Чтобы не писать в комментарии, что видео не актуально, а такого, как на видео, нет и т.п.) </center>_

---
---

In [ ]:
#@title <font color='#FF8C00'> <big>⬇️ **УСТАНОВИТЬ RVC** <big><font color='#FF000'>**`(НУЖЕН ТОЛЬКО ДЛЯ ТРЕНИРОВКИ И ПРОВЕРКИ МОДЕЛИ)`**</font>
from IPython.display import clear_output
from ipywidgets import Button
import os, time
from tqdm import tqdm
import subprocess

from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/dataset', exist_ok=True)
clear_output()

#@markdown Обновление папки TrainingModel <font color='#FF000'> **`(ПРИ ОБНОВЛЕНИИ ПАПКА TrainingModel ПОЛНОСТЬЮ УДАЛЯЕТСЯ С ДИСКА И ПЕРЕУСТАНАВЛИВАЕТСЯ)`**
update_repo = False #@param {type:"boolean"}
if update_repo:
    print('Обновление репозитория...')
    if os.path.exists('/content/drive/MyDrive/TrainingModel'):
        print('Удаление папки TrainingModel...')
        !rm -r /content/drive/MyDrive/TrainingModel
    else:
        print('Папка TrainingModel уже отсутствует')
    clear_output()

print('Клонирование репозитория...')
repo_url = 'https://github.com/Bebra777228/TrainVocModel'
repo_dir = '/content/drive/MyDrive/TrainingModel'
try:
    subprocess.check_call(['git', 'clone', repo_url, repo_dir], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
except subprocess.CalledProcessError as e:
    print(f"Ошибка при клонировании репозитория: {e}")
    clear_output()
    quit()

%cd /content/drive/MyDrive/TrainingModel
clear_output()

print('Установка зависимостей...')
!apt -y install -qq aria2 &> /dev/null
requirements_file = 'requirements.txt'
with open(requirements_file, 'r') as f:
    packages = f.read().split('\n')
progress_bar = tqdm(total=len(packages))
for package in packages:
    if package:
        subprocess.check_call(['pip', 'install', package], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
        progress_bar.update(1)
progress_bar.close()
clear_output()

print('Установка моделей...')
model_files = ['download_pretrained_models.py', 'download_files.py']
progress_bar = tqdm(total=len(model_files))
for model_file in model_files:
    subprocess.check_call(['python', model_file], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    progress_bar.update(1)
progress_bar.close()
clear_output()

from easy_sync import *
!rm -r /content/sample_data/

installed = True
clear_output()
Button(description="\u2714 Готово", button_style="success")


#**<font color='#FF8C00'><big><<< ТРЕНИРОВКА МОДЕЛИ**
---
---

In [ ]:
#@title <font color='#00FFFF'><big> ⛏️ **Обработка данных**
%cd /content/drive/MyDrive/TrainingModel
clear_output()
#@markdown ---
#@markdown Дайте своей модели имя:
model_name = 'Model_Name' #@param {type:"string"}
dataset_folder = '/content/dataset' #@param {type:"string"}

print("Запускаем процесс обработки данных...")
time.sleep(5)

while len(os.listdir(dataset_folder)) < 1:
    input("Ваша папка с наборами данных пуста.")
!mkdir -p ./logs/{model_name}
with open(f'./logs/{model_name}/preprocess.log','w') as f:
    print("Обрабатываем датасет...")
!python infer/modules/train/preprocess.py {dataset_folder} 40000 2 ./logs/{model_name} False 3.0 > /dev/null 2>&1
with open(f'./logs/{model_name}/preprocess.log','r') as f:
    if 'end preprocess' in f.read():
        clear_output()
    else:
        print("Ошибка предварительной обработки данных... Убедитесь, что папка с набором данных выбрана правильно.")

#@markdown ---
#@markdown Выберите метод для обработки данных:
f0method = "rmvpe_gpu" # @param ["pm", "harvest", "rmvpe", "rmvpe_gpu"] {allow-input: true}

#@markdown <small> **<big>`"pm"`</big> -** более быстрое выделение, но низкое качество речи

#@markdown <small> **<big>`"harvest"`</big> -** лучший бас, но очень медленный

#@markdown <small> **<big>`"rmvpe"`</big> -** лучшее качество и малые требования к GPU

#@markdown <small> **<big>`"rmvpe_gpu"`</big> -** лучшее качество, но требует больших затрат GPU

#@markdown ---
#@markdown Включить при ошибке **`FileNotFoundError: [Errno 2] No such file or directory: '../extract_f0_feature.log'`**
efp_fix = False #@param {type:"boolean"}
#@markdown ---
with open(f'./logs/{model_name}/extract_f0_feature.log','w') as f:
  if f0method != "rmvpe_gpu":
    !python infer/modules/train/extract/extract_f0_print.py ./logs/{model_name} 2 {f0method}
  else:
    !python infer/modules/train/extract/extract_f0_rmvpe.py 1 0 0 ./logs/{model_name} True
if efp_fix:
  !python infer/modules/train/extract_feature_print_fix.py cuda:0 1 0 ./logs/{model_name} v2 True
else:
  !python infer/modules/train/extract_feature_print.py cuda:0 1 0 ./logs/{model_name} v2 True
with open(f'./logs/{model_name}/extract_f0_feature.log','r') as f:
    if 'all-feature-done' in f.read():
        clear_output()
        display(Button(description="\u2714 Готово", button_style="success"))
    else:
        print("Ошибка предварительной обработки данных... Убедитесь, что папка с набором данных выбрана правильно.")

clear_output()

import numpy as np
import faiss

print("Тренируем индекс...")
time.sleep(5)

def train_index(exp_dir1, version19):
    exp_dir = "logs/%s" % (exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if not os.path.exists(feature_dir):
        return "请先进行特征提取!"
    listdir_res = list(os.listdir(feature_dir))
    if len(listdir_res) == 0:
        return "请先进行特征提取！"
    infos = []
    npys = []
    for name in sorted(listdir_res):
        phone = np.load("%s/%s" % (feature_dir, name))
        npys.append(phone)
    big_npy = np.concatenate(npys, 0)
    big_npy_idx = np.arange(big_npy.shape[0])
    np.random.shuffle(big_npy_idx)
    big_npy = big_npy[big_npy_idx]
    if big_npy.shape[0] > 2e5:
        infos.append("Trying doing kmeans %s shape to 10k centers." % big_npy.shape[0])
        yield "\n".join(infos)
        try:
            big_npy = (
                MiniBatchKMeans(
                    n_clusters=10000,
                    verbose=True,
                    batch_size=256 * config.n_cpu,
                    compute_labels=False,
                    init="random",
                )
                .fit(big_npy)
                .cluster_centers_
            )
        except:
            info = traceback.format_exc()
            logger.info(info)
            infos.append(info)
            yield "\n".join(infos)

    np.save("%s/total_fea.npy" % exp_dir, big_npy)
    n_ivf = min(int(16 * np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
    infos.append("%s,%s" % (big_npy.shape, n_ivf))
    yield "\n".join(infos)
    index = faiss.index_factory(256 if version19 == "v1" else 768, "IVF%s,Flat" % n_ivf)
    infos.append("training")
    yield "\n".join(infos)
    index_ivf = faiss.extract_index_ivf(index)  #
    index_ivf.nprobe = 1
    index.train(big_npy)
    faiss.write_index(
        index,
        "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )

    infos.append("adding")
    yield "\n".join(infos)
    batch_size_add = 8192
    for i in range(0, big_npy.shape[0], batch_size_add):
        index.add(big_npy[i : i + batch_size_add])
    faiss.write_index(
        index,
        "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )
    infos.append(
        "成功构建索引，added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (n_ivf, index_ivf.nprobe, exp_dir1, version19)
    )

training_log = train_index(model_name, 'v2')

for line in training_log:
    print(line)
    if 'adding' in line:
        clear_output()
        display(Button(description="\u2714 Готово", button_style="success"))

In [ ]:
#@title <font color='#00FFFF'><big>🤖 **Тренировка модели**
%cd /content/drive/MyDrive/TrainingModel
from random import shuffle
import json
import os
import pathlib
from subprocess import Popen, PIPE, STDOUT
now_dir=os.getcwd()
#@markdown ---
#@markdown Выберите, как часто сохранять модель и как долго она будет обучаться.
save_frequency = 20 # @param {type:"slider", min:5, max:50, step:5}
epochs = 300 # @param {type:"slider", min:10, max:2000, step:10}
batch_size = 15
cache = True
sample_rate='40k'

#@markdown ---
#@markdown Предварительно обученные модели:
pretrain = "Default" # @param ["Default", "Ov2Super - Для Иностранных моделей", "RIN_E3 - Для Иностранных моделей", "Snowie - Для Русских моделей"] {allow-input: true}
if pretrain=="Default":
  G_file=f'/content/pretrained_models/f0G{sample_rate}.pth'
  D_file=f'/content/pretrained_models/f0D{sample_rate}.pth'
elif pretrain=="Ov2Super - Для Иностранных моделей":
  G_file=f'/content/pretrained_models/f0Ov2Super{sample_rate}G.pth'
  D_file=f'/content/pretrained_models/f0Ov2Super{sample_rate}D.pth'
elif pretrain=="RIN_E3 - Для Иностранных моделей":
  G_file=f'/content/pretrained_models/f0_Rin_E3_{sample_rate}G.pth'
  D_file=f'/content/pretrained_models/f0_Rin_E3_{sample_rate}D.pth'
elif pretrain=="Snowie - Для Русских моделей":
  G_file=f'/content/pretrained_models/f0SnowieRuPre{sample_rate}G.pth'
  D_file=f'/content/pretrained_models/f0SnowieRuPre{sample_rate}D.pth'
check_model = "/content/drive/MyDrive/TrainingModel/logs/model_check.mp3"

print("Запускаем процесс тренировки модели...")
time.sleep(5)

def click_train(
    exp_dir1,
    sr2,
    if_f0_3,
    spk_id5,
    save_epoch10,
    total_epoch11,
    batch_size12,
    if_save_latest13,
    pretrained_G14,
    pretrained_D15,
    gpus16,
    if_cache_gpu17,
    if_save_every_weights18,
    version19,
):
    # 生成filelist
    exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if if_f0_3:
        f0_dir = "%s/2a_f0" % (exp_dir)
        f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
            & set([name.split(".")[0] for name in os.listdir(feature_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
        )
    else:
        names = set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)]) & set(
            [name.split(".")[0] for name in os.listdir(feature_dir)]
        )
    opt = []
    for name in names:
        if if_f0_3:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    f0_dir.replace("\\", "\\\\"),
                    name,
                    f0nsf_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
        else:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
    fea_dim = 256 if version19 == "v1" else 768
    if if_f0_3:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, now_dir, now_dir, spk_id5)
            )
    else:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, spk_id5)
            )
    shuffle(opt)
    with open("%s/filelist.txt" % exp_dir, "w") as f:
        f.write("\n".join(opt))

    # Replace logger.debug, logger.info with print statements
    print("Запись списка файлов завершена")
    print("Использование графических процессоров:", str(gpus16))
    if pretrained_G14 == "":
        print("Нет предварительно обученного генератора")
    if pretrained_D15 == "":
        print("Без предварительно обученного дискриминатора")
    if version19 == "v1" or sr2 == "40k":
        config_path = "configs/v1/%s.json" % sr2
    else:
        config_path = "configs/v2/%s.json" % sr2
    config_save_path = os.path.join(exp_dir, "config.json")
    if not pathlib.Path(config_save_path).exists():
        with open(config_save_path, "w", encoding="utf-8") as f:
            with open(config_path, "r") as config_file:
                config_data = json.load(config_file)
                json.dump(
                    config_data,
                    f,
                    ensure_ascii=False,
                    indent=4,
                    sort_keys=True,
                )
            f.write("\n")

    cmd = (
        'python infer/modules/train/train.py -e "%s" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s'
        % (
            exp_dir1,
            sr2,
            1 if if_f0_3 else 0,
            batch_size12,
            gpus16,
            total_epoch11,
            save_epoch10,
            "-pg %s" % pretrained_G14 if pretrained_G14 != "" else "",
            "-pd %s" % pretrained_D15 if pretrained_D15 != "" else "",
            1 if if_save_latest13 == True else 0,
            1 if if_cache_gpu17 == True else 0,
            1 if if_save_every_weights18 == True else 0,
            version19,
        )
    )
    # Use PIPE to capture the output and error streams
    p = Popen(cmd, shell=True, cwd=now_dir, stdout=PIPE, stderr=STDOUT, bufsize=1, universal_newlines=True)

    # Print the command's output as it runs
    for line in p.stdout:
        print(line.strip())

    # Wait for the process to finish
    p.wait()
    return "训练结束, 您可查看控制台训练日志或实验文件夹下的train.log"
%load_ext tensorboard
%tensorboard --logdir ./logs --port=8888
training_log = click_train(
    model_name,
    sample_rate,
    True,
    0,
    save_frequency,
    epochs,
    batch_size,
    True,
    G_file,
    D_file,
    0,
    cache,
    True,
    'v2',
)
print(training_log)

#@markdown ---

#**<font color='#FF8C00'><big><<< ПРОДОЛЖЕНИЕ ТРЕНИРОВКИ**
---
---

In [ ]:
#@title <font color='#00FFFF'><big>🤖 **Тренировка модели**
%cd /content/drive/MyDrive/TrainingModel

import numpy as np
import faiss
#@markdown ---
#@markdown Введите имя вашей модели:
model_name = 'Model_Name' #@param {type:"string"}
clear_output()
print("Подключаем индекс для продолжения тренировки...")
time.sleep(5)
def train_index(exp_dir1, version19):
    exp_dir = "logs/%s" % (exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if not os.path.exists(feature_dir):
        return "请先进行特征提取!"
    listdir_res = list(os.listdir(feature_dir))
    if len(listdir_res) == 0:
        return "请先进行特征提取！"
    infos = []
    npys = []
    for name in sorted(listdir_res):
        phone = np.load("%s/%s" % (feature_dir, name))
        npys.append(phone)
    big_npy = np.concatenate(npys, 0)
    big_npy_idx = np.arange(big_npy.shape[0])
    np.random.shuffle(big_npy_idx)
    big_npy = big_npy[big_npy_idx]
    if big_npy.shape[0] > 2e5:
        infos.append("Trying doing kmeans %s shape to 10k centers." % big_npy.shape[0])
        yield "\n".join(infos)
        try:
            big_npy = (
                MiniBatchKMeans(
                    n_clusters=10000,
                    verbose=True,
                    batch_size=256 * config.n_cpu,
                    compute_labels=False,
                    init="random",
                )
                .fit(big_npy)
                .cluster_centers_
            )
        except:
            info = traceback.format_exc()
            logger.info(info)
            infos.append(info)
            yield "\n".join(infos)

    np.save("%s/total_fea.npy" % exp_dir, big_npy)
    n_ivf = min(int(16 * np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
    infos.append("%s,%s" % (big_npy.shape, n_ivf))
    yield "\n".join(infos)
    index = faiss.index_factory(256 if version19 == "v1" else 768, "IVF%s,Flat" % n_ivf)
    infos.append("training")
    yield "\n".join(infos)
    index_ivf = faiss.extract_index_ivf(index)  #
    index_ivf.nprobe = 1
    index.train(big_npy)
    faiss.write_index(
        index,
        "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )

    infos.append("adding")
    yield "\n".join(infos)
    batch_size_add = 8192
    for i in range(0, big_npy.shape[0], batch_size_add):
        index.add(big_npy[i : i + batch_size_add])
    faiss.write_index(
        index,
        "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )
    infos.append(
        "成功构建索引，added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (n_ivf, index_ivf.nprobe, exp_dir1, version19)
    )

training_log = train_index(model_name, 'v2')

for line in training_log:
    print(line)
    if 'adding' in line:
        clear_output()
        display(Button(description="\u2714 Готово", button_style="success"))

print("Запускаем процесс тренировки модели...")
time.sleep(5)

from random import shuffle
import json
import os
import pathlib
from subprocess import Popen, PIPE, STDOUT
now_dir=os.getcwd()
#@markdown ---
#@markdown Выберите, как часто сохранять модель и как долго она будет обучаться.
save_frequency = 20 # @param {type:"slider", min:5, max:50, step:5}
epochs = 500 # @param {type:"slider", min:10, max:2000, step:10}
batch_size = 15
cache = True
sample_rate='40k'

#@markdown ---
#@markdown Предварительно обученные модели:
pretrain = "Default" # @param ["Default", "Ov2Super - Для Иностранных моделей", "RIN_E3 - Для Иностранных моделей", "Snowie - Для Русских моделей"] {allow-input: true}

if pretrain=="Default":
  G_file=f'/content/pretrained_models/f0G{sample_rate}.pth'
  D_file=f'/content/pretrained_models/f0D{sample_rate}.pth'
elif pretrain=="Ov2Super - Для Иностранных моделей":
  G_file=f'/content/pretrained_models/f0Ov2Super{sample_rate}G.pth'
  D_file=f'/content/pretrained_models/f0Ov2Super{sample_rate}D.pth'
elif pretrain=="RIN_E3 - Для Иностранных моделей":
  G_file=f'/content/pretrained_models/f0_Rin_E3_{sample_rate}G.pth'
  D_file=f'/content/pretrained_models/f0_Rin_E3_{sample_rate}D.pth'
elif pretrain=="Snowie - Для Русских моделей":
  G_file=f'/content/pretrained_models/f0SnowieRuPre{sample_rate}G.pth'
  D_file=f'/content/pretrained_models/f0SnowieRuPre{sample_rate}D.pth'

def click_train(
    exp_dir1,
    sr2,
    if_f0_3,
    spk_id5,
    save_epoch10,
    total_epoch11,
    batch_size12,
    if_save_latest13,
    pretrained_G14,
    pretrained_D15,
    gpus16,
    if_cache_gpu17,
    if_save_every_weights18,
    version19,
):
    # 生成filelist
    exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if if_f0_3:
        f0_dir = "%s/2a_f0" % (exp_dir)
        f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
            & set([name.split(".")[0] for name in os.listdir(feature_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
        )
    else:
        names = set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)]) & set(
            [name.split(".")[0] for name in os.listdir(feature_dir)]
        )
    opt = []
    for name in names:
        if if_f0_3:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    f0_dir.replace("\\", "\\\\"),
                    name,
                    f0nsf_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
        else:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
    fea_dim = 256 if version19 == "v1" else 768
    if if_f0_3:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, now_dir, now_dir, spk_id5)
            )
    else:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, spk_id5)
            )
    shuffle(opt)
    with open("%s/filelist.txt" % exp_dir, "w") as f:
        f.write("\n".join(opt))

    # Replace logger.debug, logger.info with print statements
    print("Запись списка файлов завершена")
    print("Использование графических процессоров:", str(gpus16))
    if pretrained_G14 == "":
        print("Нет предварительно обученного генератора")
    if pretrained_D15 == "":
        print("Без предварительно обученного дискриминатора")
    if version19 == "v1" or sr2 == "40k":
        config_path = "configs/v1/%s.json" % sr2
    else:
        config_path = "configs/v2/%s.json" % sr2
    config_save_path = os.path.join(exp_dir, "config.json")
    if not pathlib.Path(config_save_path).exists():
        with open(config_save_path, "w", encoding="utf-8") as f:
            with open(config_path, "r") as config_file:
                config_data = json.load(config_file)
                json.dump(
                    config_data,
                    f,
                    ensure_ascii=False,
                    indent=4,
                    sort_keys=True,
                )
            f.write("\n")

    cmd = (
        'python infer/modules/train/train.py -e "%s" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s'
        % (
            exp_dir1,
            sr2,
            1 if if_f0_3 else 0,
            batch_size12,
            gpus16,
            total_epoch11,
            save_epoch10,
            "-pg %s" % pretrained_G14 if pretrained_G14 != "" else "",
            "-pd %s" % pretrained_D15 if pretrained_D15 != "" else "",
            1 if if_save_latest13 == True else 0,
            1 if if_cache_gpu17 == True else 0,
            1 if if_save_every_weights18 == True else 0,
            version19,
        )
    )
    # Use PIPE to capture the output and error streams
    p = Popen(cmd, shell=True, cwd=now_dir, stdout=PIPE, stderr=STDOUT, bufsize=1, universal_newlines=True)

    # Print the command's output as it runs
    for line in p.stdout:
        print(line.strip())

    # Wait for the process to finish
    p.wait()
    return "训练结束, 您可查看控制台训练日志或实验文件夹下的train.log"
%load_ext tensorboard
%tensorboard --logdir ./logs --port=8888
training_log = click_train(
    model_name,
    sample_rate,
    True,
    0,
    save_frequency,
    epochs,
    batch_size,
    True,
    G_file,
    D_file,
    0,
    cache,
    True,
    'v2',
)
print(training_log)

#@markdown ---

# <small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small>.</small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small>

<font color='#FFFF00'>

**<center><big><big><big><big>Модель сохраняется на диск автоматически</center>**

<font color='#00FF00'>

<center> Путь к .pth файлу: </center>

<font color='#00FFFF'>

**<big><center><small> `TrainingModel/assets/weights/[имя модели].pth` </small></center></big>**

<font color='#00FF00'>

<center> Путь к .index файлу: </center>

<font color='#00FFFF'>

**<big><center><small> `TrainingModel/logs/[имя модели]/added_IVF[id]_Flat_nprobe_1_[имя модели]_v2.index` </small></center></big>**

# <small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small>.</small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small>

#**<font color='#FF8C00'><big><<< ПРОВЕРКА МОДЕЛИ**
---
---

In [ ]:
#@title <font color='#00FFFF'><big>⬇️ **Запустите перед проверкой**

!pip install python-dotenv &> /dev/null
!pip install torch torchvision torchaudio &> /dev/null

clear_output()
Button(description="\u2714 Готово", button_style="success")

In [ ]:
#@title <font color='#00FFFF'><big>🗣️ **Проверка модели**
from IPython.display import clear_output, Audio, display, HTML
from ipywidgets import Button
import fnmatch
import os, time

%cd /content
!git clone https://github.com/Bebra777228/model_check
!mkdir -p /content/output
clear_output()

%cd /content/drive/MyDrive/TrainingModel
#@markdown ---
#@markdown Введите имя модели и имя **.pth** файла для проверки его звучания _**(можно вписать сохранения)**_:
model_name = "Model_Name" #@param {type:"string"}
pth_name = "Model_Name_e120_s5280" #@param {type:"string"}
#@markdown <font color='#FF9999'> <big>**`_e120_s5280`**</big> - это точка сохранения модели. **Если вы используете полностью готовую модель, то введите в поле <big>`pth_name`</big> то же самое, что и в поле <big>`model_name`</big>.**</big>

#@markdown <big><big><font color='#FF000'>**`НЕ НУЖНО ВПИСЫВАТЬ ФОРМАТ .pth`**</font>

#@markdown ---
#@markdown Выберите метод обработки голоса:
f0_method = "rmvpe" # @param ["rmvpe", "pm", "harvest"] {allow-input: true}
#@markdown Отрегулируйте высоту тона голоса ( -12 - на октаву ниже, 0 - сохраняет исходную высоту, 12 - на октаву выше ):
transpose = 0 # @param {type:"slider", min:-12, max:12, step:1}
#@markdown ---
#@markdown **Дополнительные настройки:**

index_rate = 0.66 # @param {type:"slider", min:0, max:1, step:0.01}
volume_normalization = 0.5 # @param {type:"slider", min:0, max:1, step:0.01}
consonant_protection = 0.5 # @param {type:"slider", min:0, max:1, step:0.01}

!rm /content/output/output.wav

#@markdown <font color='#FF9999'>**Поставьте галочку если вылезла ошибка, после этого скиньте скрин ошибки в тг https://t.me/Politrees2**
check_error = False  #@param {type:"boolean"}
if check_error:
  clean_dev = ""
  clear_output()
  print("Просмотр выходных данных")
  time.sleep(5)
  print('')
  print('')
  print('')
else:
  clean_dev = "&> /dev/null"
  clear_output()
  print('Заменяем голос...')
  print('')
  print('')
  print('')

!python tools/infer_cli.py --f0up_key {transpose} --input_path /content/model_check/model_check.mp3 --index_path logs/{model_name}/added_*_v2.index --f0method {f0_method} --opt_path /content/output/output.wav --model_name {pth_name}.pth --index_rate {index_rate} --device cuda:0 --is_half True --filter_radius 3 --resample_sr 0 --rms_mix_rate {volume_normalization} --protect {consonant_protection} {clean_dev}

def find_latest_wav_file(path, pattern='*.wav'):
    matches = []

    for root, dirnames, filenames in os.walk(path):
        for filename in fnmatch.filter(filenames, pattern):
            matches.append(os.path.join(root, filename))

    if matches:
        latest_file = max(matches, key=os.path.getctime)
        return os.path.splitext(latest_file)[0]
    else:
        return None

path = '/content/output'
latest_wav_file = find_latest_wav_file(path)

if latest_wav_file:
    clear_output()
    Button(description="\u2714 Готово", button_style="success")
    NAME = latest_wav_file
    print(f'Голос успешно заменен:')
    display(Audio(f"{NAME}.wav", rate=44100))
else:
    print('Ошибка! Возможно, имя модели указано неправильно, либо в имени вокала используются недопустимые символы.')
    display(HTML("<h3 style='color:red;'>Поставьте галочку на пункт check_error, чтобы посмотреть вывод данных, который покажет, в чем проблема. Скиньте скрин ошибки в тг <a href='https://t.me/Politrees2'>https://t.me/Politrees2</a></h3>"))
    display(HTML("<h2 style='color:red;'>БЕЗ ВКЛЮЧЕННОЙ ГАЛОЧКИ ВАМ НИКТО НЕ ПОМОЖЕТ!!!</h2>"))


#**<font color='#FF8C00'><big><<< ГЕНЕРАЦИЯ КАВЕРОВ**
---
---

In [ ]:
#@title <font color='#00FFFF'><big>🚀 **УСТАНОВКА**
from IPython.display import clear_output
from ipywidgets import Button
import os, urllib
HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import runSh

clear_output()
print('Запуск...')

!git clone https://github.com/Bebra777228/TrainVocModel /content/CoverGen &> /dev/null
%cd /content/CoverGen
clear_output()
print("Установка зависимостей...")
!pip install ffmpeg-python av &> /dev/null
!pip install python-dotenv &> /dev/null
!pip install -r requirements.txt &> /dev/null
!pip install torch torchvision torchaudio &> /dev/null
clear_output()
print("Почти готово...")
!wget https://huggingface.co/Rejekts/project/resolve/main/download_files.py &> /dev/null
!python download_files.py &> /dev/null
!mkdir -p /content/output
!rm -r /content/sample_data/
clear_output()
installed = True
Button(description="\u2714 Готово", button_style="success")

In [ ]:
#@title <font color='#00FFFF'><big>🔊 **Загрузка файла с вокалом**
#@markdown Для правильной работы загрузите файлы .mp3 / .wav
from IPython.display import clear_output
from google.colab import files
from ipywidgets import Button
import os
%cd /content
!rm /content/*.*
audio = files.upload()
clear_output()

ext = os.path.splitext(list(audio.keys())[-1])[-1]
os.rename(list(audio.keys())[-1], "vocal{}".format(ext))
input_audio = "vocal" + ext
clear_output()
Button(description="\u2714 Готово", button_style="success")

In [ ]:
#@title <font color='#00FFFF'><big>🔎 **Копирование модели**
from IPython.display import clear_output
from ipywidgets import Button
import sys, os, urllib.request, time, subprocess, contextlib

#@markdown <br><center><img src='https://mega.nz/favicon.ico?v=3' height="50" alt="MEGA-logo"/></center>
#@markdown <center><h2>Копирование модели из MEGA</h2></center><br>
!mkdir -p /content/CoverGen/zip_files
!mkdir -p /content/CoverGen/assets/weights
!rm /content/CoverGen/zip_files/*
clear_output()

#@markdown Введите ссылку на ZIP архив
URL = "" #@param {type:"string"}
OUTPUT_PATH = "/content/CoverGen/zip_files"
if not OUTPUT_PATH:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"
# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Установка MEGA ...")
    runSh('apt -y update')
    runSh('wget https://mega.nz/linux/repo/xUbuntu_22.04/amd64/megacmd-xUbuntu_22.04_amd64.deb -P /var/cache/apt/archives', output=True)
    runSh('apt install "/var/cache/apt/archives/megacmd-xUbuntu_22.04_amd64.deb"', output=True)
    print("MEGA установлена.")
    clear_output()

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", URL, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)



transfare()

!unzip -n /content/CoverGen/zip_files/* -d /content/CoverGen/assets/weights

%cd /content/CoverGen/assets/weights
clear_output()

print("Установленные модели:")
print("")
!ls -C -w 10 -u *.pth

#@markdown ---
#@markdown **Инструкция**

#@markdown 1. Заходим на сайт https://mega.nz и регаемся там *( Дается 25гб, этого хватит за глаза )*

#@markdown 2. Закидываем туда ZIP файл с моделью *( .pth / .index файлы )*

#@markdown 3. Нажимаем на 3 точки > Поделиться ссылокой

#@markdown 4. Вставляем эту ссылку сюда

In [ ]:
#@title <font color='#00FFFF'><big>🗣️ **Замена голоса**
from IPython.display import clear_output, Audio, display, HTML
from ipywidgets import Button
import fnmatch
import os, time
%cd /content/CoverGen
#@markdown ---
#@markdown Введите имя модели и имя **.pth** файла **(можно вписать сохранения)**:
model_name = "Model_Name" #@param {type:"string"}
pth_name = "Model_Name_e120_s5280" #@param {type:"string"}
#@markdown <font color='#FF9999'> <big>**`_e120_s5280`**</big> - это точка сохранения модели. **Если вы используете полностью готовую модель, то введите в поле <big>`pth_name`</big> то же самое, что и в поле <big>`model_name`</big>.**</big>

#@markdown <big><big><font color='#FF000'>**`НЕ НУЖНО ВПИСЫВАТЬ ФОРМАТ .pth`**</font>

#@markdown ---
#@markdown Выберите метод обработки голоса:
f0_method = "rmvpe" # @param ["rmvpe", "pm", "harvest"] {allow-input: true}
#@markdown Отрегулируйте высоту тона голоса ( -12 - на октаву ниже, 0 - сохраняет исходную высоту, 12 - на октаву выше ):
transpose = 0 # @param {type:"slider", min:-12, max:12, step:1}
#@markdown ---
#@markdown **Дополнительные настройки:**

index_rate = 0.66 # @param {type:"slider", min:0, max:1, step:0.01}
volume_normalization = 0.5 # @param {type:"slider", min:0, max:1, step:0.01}
consonant_protection = 0.5 # @param {type:"slider", min:0, max:1, step:0.01}

!rm /content/output/output.wav

#@markdown <font color='#FF9999'>**Поставьте галочку если вылезла ошибка, после этого скиньте скрин ошибки в тг https://t.me/Politrees2**
check_error = False  #@param {type:"boolean"}
if check_error:
  clean_dev = ""
  clear_output()
  print("Просмотр выходных данных")
  time.sleep(5)
  print('')
  print('')
  print('')
else:
  clean_dev = "&> /dev/null"
  clear_output()
  print('Заменяем голос...')
  print('')
  print('')
  print('')

!python tools/infer_cli.py --f0up_key {transpose} --input_path /content/*.* --index_path assets/weights/*{model_name}_v2.index --f0method {f0_method} --opt_path /content/output/output.wav --model_name {pth_name}.pth --index_rate {index_rate} --device cuda:0 --is_half True --filter_radius 3 --resample_sr 0 --rms_mix_rate {volume_normalization} --protect {consonant_protection} {clean_dev}

def find_latest_wav_file(path, pattern='*.wav'):
    matches = []

    for root, dirnames, filenames in os.walk(path):
        for filename in fnmatch.filter(filenames, pattern):
            matches.append(os.path.join(root, filename))

    if matches:
        latest_file = max(matches, key=os.path.getctime)
        return os.path.splitext(latest_file)[0]
    else:
        return None

path = '/content/output'
latest_wav_file = find_latest_wav_file(path)

if latest_wav_file:
    clear_output()
    Button(description="\u2714 Готово", button_style="success")
    NAME = latest_wav_file
    print(f'Голос успешно заменен:')
    display(Audio(f"{NAME}.wav", rate=44100))
else:
    print('Ошибка! Возможно, имя модели указано неправильно, либо в имени вокала используются недопустимые символы.')
    display(HTML("<h3 style='color:red;'>Поставьте галочку на пункт check_error, чтобы посмотреть вывод данных, который покажет, в чем проблема. Скиньте скрин ошибки в тг <a href='https://t.me/Politrees2'>https://t.me/Politrees2</a></h3>"))
    display(HTML("<h2 style='color:red;'>БЕЗ ВКЛЮЧЕННОЙ ГАЛОЧКИ ВАМ НИКТО НЕ ПОМОЖЕТ!!!</h2>"))


<font color='#00FFFF'>

# <small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small>.</small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small>

**[<center><big><big><big><big><big> Оригинальный блокнот </center>](https://colab.research.google.com/drive/1r4IRL0UA7JEoZ0ZK8PKfMyTIBHKpyhcw?usp=sharing)**
# <small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small>.</small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small>

---
---

# <small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small>.</small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small>
**<center><big><big> ПРОЧИЕ ПРИКОЛЮХИ </center>**
<center> ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓ </center>

# <small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small>.</small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small>


#**<font color='#FF8C00'><big><<< АВТОМАТИЧЕСКАЯ ГЕНЕРАЦИЯ КАВЕРОВ**
<big> **[Тутор по ячейке](https://youtu.be/ysr7MQwzfT8)**

---
---

In [ ]:
#@title <font color='#00FFFF'><big>🚀 **УСТАНОВКА**
from IPython.display import clear_output
from ipywidgets import Button
import os, urllib.request

HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import runSh

print('Запуск...')


# Обфускация
hug_user = "https"+"://hugg"+"ingface.co/seang"+"hay/uv"+"r_models/res"+"olve/main/"
a_hug_model = "9_HP"+"2-UV"+"R.p"+"th"
b_hug_model = "UV"+"R-DeE"+"cho-DeRe"+"verb.p"+"th"

# Создание необходимых папок
!mkdir -p /content/input
!mkdir -p /content/output
!mkdir -p /content/output/ouvr
!mkdir -p /content/output/processed_rvc

# Установка репозиториев
!git clone https://github.com/Bebra777228/TrainVocModel /content/CoverGen  &> /dev/null
!git clone https://github.com/jarredou/MVSEP-MDX23-Colab_v2 /content/CoverGen/UVR  &> /dev/null

# Установка моделей
%cd /content/CoverGen
clear_output()
print('Установка моделей...')
!wget https://huggingface.co/Rejekts/project/resolve/main/download_files.py &> /dev/null
!python download_files.py &> /dev/null
!wget {hug_user}{a_hug_model} -P /content/CoverGen/assets/uvr5_weights/  &> /dev/null
!wget {hug_user}{b_hug_model} -P /content/CoverGen/assets/uvr5_weights/  &> /dev/null

# Установка зависимостей
%cd /content/CoverGen/lib
clear_output()
print('Установка зависимостей...')
!pip install ffmpeg-python av &> /dev/null
!pip install python-dotenv &> /dev/null
!pip install -r requirements.txt &> /dev/null
!pip install torch torchvision torchaudio &> /dev/null

impulse_folder = '/content/CoverGen/impulse/VS8F-1'
impulse_filename = '100-Reverb.wav'
impulse_file = os.path.join(impulse_folder, impulse_filename)

# Удаление ненужных папок
!rm -r /content/sample_data/

clear_output()
installed = True
Button(description="\u2714 Готово", button_style="success")

In [ ]:
#@title <font color='#00FFFF'><big>✂️ **РАЗДЕЛЕНИЕ**
#@markdown ---

#@markdown **МАКСИМАЛЬНОЕ ВРЕМЯ ОБРАБОТКИ 30 МИНУТ** *(зависит от длины трека и **quaility** значения)*
from IPython.display import clear_output, Audio, display, HTML, FileLink
from ipywidgets import Button
from google.colab import files
from pathlib import Path, PurePath
import os, time, ffmpeg

# --------------------
# Загрузка файла песни
# --------------------

%cd /content/input
!rm /content/input/*.*
!rm /content/output/ouvr/*.*
clear_output()
audio = files.upload()

ext = os.path.splitext(list(audio.keys())[-1])[-1]
os.rename(list(audio.keys())[-1], "Music{}".format(ext))
input_audio = "Music" + ext
clear_output()

# ---------------------------------
# Отделение вокала от инструментала
# ---------------------------------

%cd /content/CoverGen/UVR

input = '/content/input'
output_uvr_folder = '/content/output/ouvr'
#@markdown ---
#@markdown Соотношение **скорость** / **качество** (1 - максимальная скорость, 10 - максимальное качество):
quaility = 5 # @param {type:"slider", min:1, max:10, step:1}
quality_max = 10

BigShifts = round(11 * quaility / quality_max)
overlap_InstVoc = round(8 * quaility / quality_max)
overlap_VitLarge = round(8 * quaility / quality_max)
weight_InstVoc = 8
weight_VitLarge = 5
is_VOCFT = False
overlap_VOCFT = 0.1
weight_VOCFT = 1
is_vocals_only = True
overlap_demucs = 0.6
output_format = 'PCM_16'
vocals_only = ''
use_VOCFT = ''
if is_vocals_only:
    vocals_only = '--vocals_only true'
else:
    vocals_only = ''
if is_VOCFT:
    use_VOCFT = '--use_VOCFT true'
else:
    use_VOCFT = ''


filename = next(Path(input).glob('*.mp3'))
input_name = Path(filename).stem
vocal_file = os.path.join(output_uvr_folder, input_name + "_vocals.wav")
instrum_file = os.path.join(output_uvr_folder, input_name + "_instrum.wav")

result = !python inference.py --large_gpu --weight_InstVoc {weight_InstVoc} --weight_VOCFT {weight_VOCFT} --weight_VitLarge {weight_VitLarge} --input_audio "{filename}" --overlap_demucs {overlap_demucs} --overlap_VOCFT {overlap_VOCFT} --overlap_InstVoc {overlap_InstVoc} --overlap_VitLarge {overlap_VitLarge} --output_format {output_format} --BigShifts {BigShifts} --output_folder "{output_uvr_folder}" {vocals_only} {use_VOCFT}

if len(result) > 0:
    print("\nВывод скрипта вывода UVR:")
    for line in result:
        print(line)

# ---------------------------------
# Небольшая обработка (скоро будет)
# ---------------------------------

clear_output()
Button(description="\u2714 Готово", button_style="success")

original = "/content/input/Music.mp3"
vocals = "/content/output/ouvr/Music_vocals.wav"
instrumental = "/content/output/ouvr/Music_instrum.wav"
output_vocals = "/content/output/ouvr/Music_vocals.mp3"
output_instrumental = "/content/output/ouvr/Music_instrum.mp3"

if not os.path.exists(vocals) or not os.path.exists(instrumental):
    print("Ошибка: Файлы вокала и инструментала не были сгенерированы корректно.")
else:
    print("\n\nФайлы вокала и инструментала успешно сгенерированы.")

    input_voc = ffmpeg.input(vocals)
    output_voc = ffmpeg.output(input_voc, output_vocals)

    input_inst = ffmpeg.input(instrumental)
    output_inst = ffmpeg.output(input_inst, output_instrumental)

    print("ОРИГИНАЛ:")
    display(Audio(original, autoplay=False))
    time.sleep(2)
    print("\n\n\nВОКАЛ:")
    time.sleep(1)
    try:
        ffmpeg.run(output_voc)
    except ffmpeg.Error as e:
        print("Ошибка при конвертации вокала:\n")
        print(e.stderr.decode())
    display(Audio(output_vocals, autoplay=False))
    time.sleep(2)
    print("\n\nИНСТРУМЕНТАЛ:")
    time.sleep(1)
    try:
        ffmpeg.run(output_inst)
    except ffmpeg.Error as e:
        print("Ошибка при конвертации инструментала:\n")
        print(e.stderr.decode())
    display(Audio(output_instrumental, autoplay=False))

!rm /content/output/ouvr/*.mp3

In [ ]:
#@title <font color='#00FFFF'><big>🔎 **Копирование модели**
from IPython.display import clear_output
from ipywidgets import Button
import sys, os, urllib.request, time, subprocess, contextlib

#@markdown <br><center><img src='https://mega.nz/favicon.ico?v=3' height="50" alt="MEGA-logo"/></center>
#@markdown <center><h2>Копирование модели из MEGA</h2></center><br>
!mkdir -p /content/CoverGen/zip_files
!rm /content/CoverGen/zip_files/*
clear_output()

#@markdown Введите ссылку на ZIP архив
URL = "" #@param {type:"string"}
OUTPUT_PATH = "/content/CoverGen/zip_files"
if not OUTPUT_PATH:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"
# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Установка MEGA ...")
    runSh('apt -y update')
    runSh('wget https://mega.nz/linux/repo/xUbuntu_22.04/amd64/megacmd-xUbuntu_22.04_amd64.deb -P /var/cache/apt/archives', output=True)
    runSh('apt install "/var/cache/apt/archives/megacmd-xUbuntu_22.04_amd64.deb"', output=True)
    print("MEGA установленна.")
    clear_output()

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", URL, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)



transfare()

!unzip -n /content/CoverGen/zip_files/* -d /content/CoverGen/assets/weights

%cd /content/CoverGen/assets/weights
clear_output()

print("Установленные модели:")
print("")
!ls -C -w 10 -u *.pth

#@markdown ---
#@markdown **Инструкция**

#@markdown 1. Заходим на сайт https://mega.nz и регаемся там *( Дается 25гб, этого хватит за глаза )*

#@markdown 2. Закидываем туда ZIP файл с моделью *( .pth / .index файлы )*

#@markdown 3. Нажимаем на 3 точки > Поделиться ссылокой

#@markdown 4. Вставляем эту ссылку сюда

In [ ]:
#@title <font color='#00FFFF'><big>🗣️  **Генерация кавера**
from IPython.display import clear_output, Audio, display, HTML, FileLink
from ipywidgets import Button
import os, fnmatch, shutil, time
%cd /content/CoverGen
#@markdown ---
#@markdown Введите имя модели и имя **.pth** файла **(можно вписать сохранения)**:
model_name = "Model_Name" #@param {type:"string"}
pth_name = "Model_Name_e120_s5280" #@param {type:"string"}
#@markdown <font color='#FF9999'> <big>**`_e120_s5280`**</big> - это точка сохранения модели. **Если вы используете полностью готовую модель, то введите в поле <big>`pth_name`</big> то же самое, что и в поле <big>`model_name`</big>.**</big>

#@markdown <big><big><font color='#FF000'>**`НЕ НУЖНО ВПИСЫВАТЬ ФОРМАТ .pth`**</font>

#@markdown ---
#@markdown Выберите метод обработки голоса:
f0_method = "rmvpe" # @param ["rmvpe", "pm", "harvest"] {allow-input: true}
#@markdown Отрегулируйте высоту тона голоса ( -12 - на октаву ниже, 0 - сохраняет исходную высоту, 12 - на октаву выше ):
transpose = 0 # @param {type:"slider", min:-12, max:12, step:1}
#@markdown ---
#@markdown **Дополнительные настройки:**

index_rate = 0.66 # @param {type:"slider", min:0, max:1, step:0.01}
volume_normalization = 0.5 # @param {type:"slider", min:0, max:1, step:0.01}
consonant_protection = 0.5 # @param {type:"slider", min:0, max:1, step:0.01}

!rm /content/output/*.*
!rm /content/output/processed_rvc/*.*


#@markdown <font color='#FF9999'>**Поставьте галочку если вылезла ошибка, после этого скиньте скрин ошибки в тг https://t.me/Politrees2**
check_error = False  #@param {type:"boolean"}
if check_error:
  clean_dev = ""
  clear_output()
  print("Просмотр выходных данных")
  time.sleep(5)
  print('')
  print('')
  print('')
else:
  clean_dev = "&> /dev/null"
  clear_output()
  print('Заменяем голос...')
  print('')
  print('')
  print('')

!python tools/infer_cli.py --f0up_key {transpose} --input_path /content/output/ouvr/*_vocals.wav --index_path assets/weights/*{model_name}_v2.index --f0method {f0_method} --opt_path /content/output/processed_rvc/output.wav --model_name {pth_name}.pth --index_rate {index_rate} --device cuda:0 --is_half True --filter_radius 3 --resample_sr 0 --rms_mix_rate {volume_normalization} --protect {consonant_protection} {clean_dev}

# ----------------
# Обработка вокала
# ----------------

impulse_folder = '/content/CoverGen/impulse/VS8F-1'
impulse_filename = '100-Reverb.wav'
impulse_file = os.path.join(impulse_folder, impulse_filename)
rvc_result_file = '/content/output/processed_rvc/output.wav'
output_path = '/content/output/processed_rvc'
processed_output_format = 'mp3'
compressed_result_file = os.path.join(output_path, f"{os.path.splitext(rvc_result_file)[0]}_compressed.{processed_output_format}")
processed_result_file = os.path.join(output_path, f"{os.path.splitext(rvc_result_file)[0]}_processed.{processed_output_format}")
print(f"Добавление эффектов на голос...")
!ffmpeg -y -i {rvc_result_file} -filter_complex "anlmdn=s=10,acompressor=threshold=-20dB:ratio=4:attack=20:release=200,volume=2,loudnorm=I=-13:TP=-1.0:LRA=9,volume=1.5" {compressed_result_file}
if os.path.isfile(compressed_result_file) != True:
  print(f"Не удалось обработать файл {rvc_result_file}")
  shutil.copy(rvc_result_file, processed_result_file)
else:
  if os.path.isfile(impulse_file):
    print("Добавление реверберации с разной обработкой для левого и правого канала для стереоскопического эффекта")
    !ffmpeg -y -i {compressed_result_file} -i {impulse_file} -filter_complex "[0:a]asplit=2[splita][splitb]; [splita]adelay=40|40[splita_delayed]; [splitb]adelay=20|20[splitb_delayed]; [splita_delayed][1]afir=dry=10:wet=10[reverb_left]; [splitb_delayed][1]afir=dry=10:wet=10[reverb_right]; [reverb_left][reverb_right]amerge=inputs=2[reverb]; [0:a][reverb]amix=inputs=2:weights=20 1[audio]" -map "[audio]" {processed_result_file}
    if os.path.isfile(processed_result_file):
      !rm -rf {compressed_result_file}
    else:
      print(f"Не удалось обработать сжатый файл {compressed_result_file}")
      shutil.move(compressed_result_file, processed_result_file)
  else:
    print(f"Импульсный файл не найден: {impulse_file}")
    shutil.move(compressed_result_file, processed_result_file)
clear_output()

# ----------------------------------
# Соединение вокала с инструменталом
# ----------------------------------

output_path = '/content/output'
output_format = 'mp3'

result_file = os.path.join(output_path, input_name + "." + output_format)

!ffmpeg -y -i {processed_result_file} -i {instrum_file} -filter_complex "[0:a][1:a]amerge=inputs=2[a]" -map "[a]" -ac 2 {result_file}

clear_output()
Button(description="\u2714 Готово", button_style="success")

audio = Audio(result_file, autoplay=False)
display(audio)

In [ ]:
#@title 💾  **Скачивание файлов**

#@markdown ---
#@markdown #<big> UVR

Download_UVR = "None" # @param ["Вокал", "Инструментал", "Вокал & Инструментал"] {allow-input: true}

if Download_UVR=="Вокал":
  print('Скачиваем Вокал...')
  files.download('/content/output/ouvr/Music_vocals.wav')
  time.sleep(5)
elif Download_UVR=="Инструментал":
  print('Скачиваем Инструментал...')
  files.download('/content/output/ouvr/Music_instrum.wav')
  time.sleep(5)
elif Download_UVR=="Вокал & Инструментал":
  print('Скачиваем Вокал и Инструментал...')
  files.download('/content/output/ouvr/Music_vocals.wav')
  files.download('/content/output/ouvr/Music_instrum.wav')
  time.sleep(5)

#@markdown ---
#@markdown #<big> Ai Cover

Download_Ai_Cover = "None" # @param ["Обработанный Вокал", "Готовый Ai Cover"] {allow-input: true}

if Download_Ai_Cover=="Обработанный Вокал":
  print('Скачиваем Обработанный Вокал...')
  files.download('/content/output/processed_rvc/output.wav')
  time.sleep(5)
elif Download_Ai_Cover=="Готовый Ai Cover":
  print('Скачиваем Ai Cover...')
  files.download('/content/output/Music.mp3')
  time.sleep(5)

#@markdown ---

#**<font color='#FF8C00'><big><<< UVR** <small><small><small> ***(1 вариант)***
---
---

In [ ]:
#@title <font color='#00FFFF'><big>🚀 **УСТАНОВКА UVR**
from IPython.display import clear_output
from ipywidgets import Button
import os

# Создание необходимых папок
!mkdir -p /content/input
!mkdir -p /content/output

# Обфускация
hug_user = "https"+"://hugg"+"ingface.co/seang"+"hay/uv"+"r_models/res"+"olve/main/"
a_hug_model = "9_HP"+"2-UV"+"R.p"+"th"
b_hug_model = "UV"+"R-DeE"+"cho-DeRe"+"verb.p"+"th"

# Установка UVR и моделей
!git clone https://github.com/jarredou/MVSEP-MDX23-Colab_v2  &> /dev/null /content/UVR
!git clone https://github.com/Bebra777228/m_param  &> /dev/null /content/UVR/lib
%cd /content/UVR
clear_output()
print('Идет установка, подождите!')
!pip install ffmpeg-python &> /dev/null
!pip install -r requirements.txt &> /dev/null
!wget {hug_user}{a_hug_model} -P /content/UVR/assets/uvr5_weights/  &> /dev/null
!wget {hug_user}{b_hug_model} -P /content/UVR/assets/uvr5_weights/  &> /dev/null
!python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/   &> /dev/null
!pip install https://github.com/karaokenerds/python-audio-separator/releases/download/v0.12.1/onnxruntime_gpu-1.17.0-cp310-cp310-linux_x86_64.whl &> /dev/null

# Уделаение ненужных папок
!rm -r /content/sample_data/

clear_output()
Button(description="\u2714 Готово", button_style="success")

In [ ]:
#@title <font color='#00FFFF'><big>✂️ **РАЗДЕЛЕНИЕ**
#@markdown ---

#@markdown **МАКСИМАЛЬНОЕ ВРЕМЯ ОБРАБОТКИ 30 МИНУТ** *(зависит от длины трека и **quaility** значения)*
from IPython.display import clear_output, Audio, display, HTML, FileLink
from ipywidgets import Button
from google.colab import files
from pathlib import Path, PurePath
import os, time, ffmpeg

# --------------------
# Загрузка файла песни
# --------------------

%cd /content/input
!rm /content/input/*.*
!rm /content/output/*.*
clear_output()
audio = files.upload()

ext = os.path.splitext(list(audio.keys())[-1])[-1]
os.rename(list(audio.keys())[-1], "Music{}".format(ext))
input_audio = "Music" + ext
clear_output()

# ---------------------------------
# Отделение вокала от инструментала
# ---------------------------------

%cd /content/UVR

input = '/content/input'
output_uvr_folder = '/content/output'
#@markdown ---
#@markdown Соотношение **скорость** / **качество** (1 - максимальная скорость, 10 - максимальное качество):
quaility = 5 # @param {type:"slider", min:1, max:10, step:1}
quality_max = 10
#@markdown ---

BigShifts = round(11 * quaility / quality_max)
overlap_InstVoc = round(8 * quaility / quality_max)
overlap_VitLarge = round(8 * quaility / quality_max)
weight_InstVoc = 8
weight_VitLarge = 5
is_VOCFT = False
overlap_VOCFT = 0.1
weight_VOCFT = 1
is_vocals_only = True
overlap_demucs = 0.6
output_format = 'PCM_16'
vocals_only = ''
use_VOCFT = ''
if is_vocals_only:
    vocals_only = '--vocals_only true'
else:
    vocals_only = ''
if is_VOCFT:
    use_VOCFT = '--use_VOCFT true'
else:
    use_VOCFT = ''


filename = next(Path(input).glob('*.mp3'))
input_name = Path(filename).stem
vocal_file = os.path.join(output_uvr_folder, input_name + "_vocals.wav")
instrum_file = os.path.join(output_uvr_folder, input_name + "_instrum.wav")

!python inference.py --large_gpu --weight_InstVoc {weight_InstVoc} --weight_VOCFT {weight_VOCFT} --weight_VitLarge {weight_VitLarge} --input_audio "{filename}" --overlap_demucs {overlap_demucs} --overlap_VOCFT {overlap_VOCFT} --overlap_InstVoc {overlap_InstVoc} --overlap_VitLarge {overlap_VitLarge} --output_format {output_format} --BigShifts {BigShifts} --output_folder "{output_uvr_folder}" {vocals_only} {use_VOCFT}

# ---------------------------------
# Небольшая обработка (скоро будет)
# ---------------------------------

clear_output()
Button(description="\u2714 Готово", button_style="success")

original = "/content/input/Music.mp3"
vocals = "/content/output/Music_vocals.wav"
intrumental = "/content/output/Music_instrum.wav"
output_vocals = "/content/output/Music_vocals.mp3"
output_intrumental = "/content/output/Music_instrum.mp3"

input_voc = ffmpeg.input(vocals)
output_voc = ffmpeg.output(input_voc, output_vocals)

input_inst = ffmpeg.input(intrumental)
output_inst = ffmpeg.output(input_inst, output_intrumental)

print("ОРИГИНАЛ:")
display(Audio(original, autoplay=False))
time.sleep(2)
print("")
print("")
print("")
print("ВОКАЛ:")
time.sleep(1)
ffmpeg.run(output_voc)
display(Audio(output_vocals, autoplay=False))
time.sleep(2)
print("")
print("ИНСТРУМЕНТАЛ:")
time.sleep(1)
ffmpeg.run(output_inst)
display(Audio(output_intrumental, autoplay=False))

!rm /content/output/*.wav

In [ ]:
#@title <font color='#00FFFF'><big>💾  **Скачивание файлов**
#@markdown ---
#@markdown Выберите из списка то, что хотите скачать:
Download_UVR = "None" # @param ["Вокал", "Инструментал", "Вокал & Инструментал"] {allow-input: true}

if Download_UVR=="Вокал":
  print('Скачиваем Вокал...')
  files.download('/content/output/Music_vocals.mp3')
  time.sleep(5)
elif Download_UVR=="Инструментал":
  print('Скачиваем Инструментал...')
  files.download('/content/output/Music_instrum.mp3')
  time.sleep(5)
elif Download_UVR=="Вокал & Инструментал":
  print('Скачиваем Вокал и Инструментал...')
  files.download('/content/output/Music_vocals.mp3')
  files.download('/content/output/Music_instrum.mp3')
  time.sleep(5)

#@markdown ---

#**<font color='#FF8C00'><big><<< UVR** <small><small><small> ***(2 вариант)***
---
---

In [ ]:
#@title <font color='#00FFFF'><big>🚀 **УСТАНОВКА UVR**
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs &> /dev/null

!mkdir -p /content/input
!mkdir -p /content/output

!rm -r /content/sample_data/

In [ ]:
#@title <font color='#00FFFF'><big>🔊 **Загрузка песни**
from IPython.display import clear_output
from google.colab import files
from ipywidgets import Button
import os

%cd /content/input
!rm /content/input/*.*
clear_output()

audio = files.upload()
clear_output()

ext = os.path.splitext(list(audio.keys())[-1])[-1]
os.rename(list(audio.keys())[-1], "music{}".format(ext))
input_audio = "music" + ext

clear_output()
Button(description="\u2714 Готово", button_style="success")

In [ ]:
#@title <font color='#00FFFF'><big>✂️ **РАЗДЕЛЕНИЕ**
#@markdown ---
#@markdown Модели:
model = "mdx" #@param ["htdemucs", "htdemucs_ft", "htdemucs_6s", "hdemucs_mmi", "mdx", "mdx_extra"] {allow-input: true}
extensions = ["mp3", "wav", "ogg", "flac"]

#@markdown Поставьте галочку чтоб разделить песню на <big> _**`vocals / bass / drums / other `**_
v_b_d_o = False #@param {type:"boolean"}
if v_b_d_o:
  two_stems = None
else:
  two_stems = "vocals"

mp3 = True
mp3_rate = 320
float32 = False
int24 = False

in_path = '/content/input/'
out_path = '/content/output/'

#@markdown ---

import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO
import time
import os.path
from IPython.display import Audio, display

from google.colab import files

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


def from_upload():
    out_path = Path('separated')
    in_path = Path('tmp_in')

    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()

    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()

    uploaded = files.upload()
    for name, content in uploaded.items():
        (in_path / name).write_bytes(content)
    separate(in_path, out_path)

separate()

clear_output()
Button(description="\u2714 Готово", button_style="success")

original = "/content/input/music.mp3"
out_vocals = os.path.join("/content/output/" + model + "/music/vocals.mp3")
out_intrumental = os.path.join("/content/output/" + model + "/music/no_vocals.mp3")
out_bass = os.path.join("/content/output/" + model + "/music/bass.mp3")
out_drums = os.path.join("/content/output/" + model + "/music/drums.mp3")
out_other = os.path.join("/content/output/" + model + "/music/other.mp3")

if v_b_d_o:
  print("ОРИГИНАЛ:")
  display(Audio(original, autoplay=False))
  time.sleep(2)
  print("")
  print("")
  print("")
  print("ВОКАЛ:")
  time.sleep(1)
  display(Audio(out_vocals, autoplay=False))
  time.sleep(2)
  print("")
  print("БАСС:")
  time.sleep(1)
  display(Audio(out_bass, autoplay=False))
  time.sleep(2)
  print("")
  print("БАРАБАНЫ:")
  time.sleep(1)
  display(Audio(out_drums, autoplay=False))
  time.sleep(2)
  print("")
  print("ДРУГОЕ:")
  time.sleep(1)
  display(Audio(out_other, autoplay=False))
else:
  print("ОРИГИНАЛ:")
  display(Audio(original, autoplay=False))
  time.sleep(2)
  print("")
  print("")
  print("")
  print("ВОКАЛ:")
  time.sleep(1)
  display(Audio(out_vocals, autoplay=False))
  time.sleep(2)
  print("")
  print("ИНСТРУМЕНТАЛ:")
  time.sleep(1)
  display(Audio(out_intrumental, autoplay=False))

In [ ]:
#@title <font color='#00FFFF'><big>💾  **Скачивание файлов**

#@markdown ---
#@markdown Файлы загружаются с учетом выбранной модели и галочки **`v_b_d_o`**

#@markdown Выберите для себя идеальный вариант **разделения**, после этого уже запускайте эту ячейку.

#@markdown ---

if v_b_d_o:
  files.download(out_vocals)
  files.download(out_bass)
  files.download(out_drums)
  files.download(out_other)
else:
  files.download(out_vocals)
  files.download(out_intrumental)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>